In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

import time
import pandas as pd

## Get Tickers

In [106]:
tickers = pd.read_csv("data/brands/tickers/tickers.csv")['tic'].dropna()
tickers = tickers.str.replace('\d+', '')
tickers = tickers.str.split('.').apply(lambda x: x[0])
tickers = set(tickers.values.tolist())

## Hybrid Selenium / BeautifulSoup Approach

In [ ]:
# executable_path = 'C:/Users/Fabrice/Documents/GitHub/biz_sentiment/webscrape/drivers/chromedriver.exe'
# driver = webdriver.Chrome(executable_path=executable_path)

# ticker = 'VVUSQ'
# skip_inactive = False
# pagetypes = ['competitor', 'customer', 'supplier', 'partner']

# base_url = "https://www.mergentonline.com/"
# search_url = base_url + "basicsearch.php"

# driver.get(search_url)

# searchbox = driver.find_element_by_id("basicsearchinput")
# searchbox.clear()
# searchbox.send_keys(ticker)

# wait = WebDriverWait(driver, 5)
# wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'livesearchresult')))

# soup = BeautifulSoup(driver.page_source, 'html.parser')
# table = soup.find('table', class_='livesearchresult')

# df_table = pd.read_html(str(table))[0]
# df_table.columns = ['Ticker', 'Exchange', 'Name']

# us_exchanges = ['BZX', 'NAS','CHX', 'IEX', 'ISE', 'NMS', 'NYS']
# df_table_us = df_table[df_table['Exchange'].isin(us_exchanges)]
# df_table_us = df_table_us[df_table_us['Ticker'] == ticker]
# if skip_inactive: 
#     df_table_us = df_table_us[~df_table_us['Name'].str.contains('inactive')]
# if len(df_table_us) > 0:
#     idx = df_table_us.index[0]

# redirected=False
# supply_chain_url = base_url + 'companyhorizon.php?'
# compnumber = table.find_all('a')[idx].get('href').split('&')[0].split('?')[1]
# page = supply_chain_url + compnumber
# driver.get(page)
# redirected=True

# downloaded=False
# for pagetype in pagetypes:
#     tab = supply_chain_url + 'pagetype=' + pagetype + '&' + compnumber
#     driver.get(tab)
#     if 'no ' + pagetype + ' found' in driver.page_source.lower():
#         continue
#     driver.find_element_by_id("excellinkid").click()
#     downloaded = True


In [173]:
def mergent_supply_chain_search(driver, ticker):
    
    base_url = "https://www.mergentonline.com/"
    search_url = base_url + "horizonsearch.php"

    driver.get(search_url)

    searchbox = driver.find_element_by_id("searchtext")
    searchbox.clear()
    searchbox.send_keys(ticker)

    wait = WebDriverWait(driver, 5)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'livesearchresult')))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.find('table', class_='livesearchresult')
    if results is None:
        return False
    links = results.find_all('a')
    redirected=False
    for link in links:
        if '-US%29' in str(link):
            driver.get(base_url + link.get('href'))
            redirected=True
    if not redirected:
        # print('Could not find US company for stock ticker:', ticker)
        return False

    driver.find_element_by_id("excel").click() 
    return True

def mergent_basic_search(driver, ticker, pagetypes = ['competitor', 'customer', 'supplier', 'partner'], skip_inactive=True):
    
    base_url = "https://www.mergentonline.com/"
    search_url = base_url + "basicsearch.php"

    driver.get(search_url)

    searchbox = driver.find_element_by_id("basicsearchinput")
    searchbox.clear()
    searchbox.send_keys(ticker)

    wait = WebDriverWait(driver, 5)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'livesearchresult')))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table', class_='livesearchresult')

    df_table = pd.read_html(str(table))[0]
    df_table.columns = ['Ticker', 'Exchange', 'Name']

    us_exchanges = ['BZX', 'NAS','CHX', 'IEX', 'ISE', 'NMS', 'NYS']
    df_table_us = df_table[df_table['Exchange'].isin(us_exchanges)]
    df_table_us = df_table_us[df_table_us['Ticker'] == ticker]
    if skip_inactive: 
        df_table_us = df_table_us[~df_table_us['Name'].str.contains('inactive')]
    if len(df_table_us) > 0:
        idx = df_table_us.index[0]
    else:
        return False

    redirected=False
    supply_chain_url = base_url + 'companyhorizon.php?'
    compnumber = table.find_all('a')[idx].get('href').split('&')[0].split('?')[1]
    page = supply_chain_url + compnumber
    driver.get(page)
    redirected=True

    if not redirected:
        # print('Could not find US company for stock ticker:', ticker)
        return False
    
    downloaded=False
    for pagetype in pagetypes:
        tab = supply_chain_url + 'pagetype=' + pagetype + '&' + compnumber
        driver.get(tab)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        if not soup.find('table', class_='tablesorter bodyline'):
            continue
        driver.find_element_by_id("excellinkid").click()
        downloaded = True
        
    if not downloaded:
        return False 
    return True

## Get Brands

In [ ]:
executable_path = 'C:/Users/Fabrice/Documents/GitHub/biz_sentiment/webscrape/drivers/chromedriver.exe'
driver = webdriver.Chrome(executable_path=executable_path)

tickers = set(pd.read_csv("data/tickers/tickers.csv")['tic'].str.replace('[^a-zA-Z]', '').values.tolist())
tickers = [ticker for ticker in tickers if isinstance(ticker, str)]

print('Total tickers:', len(tickers))

passes, fails = [], []
for ticker in tickers:
    success = mergent_supply_chain_search(driver, ticker)
    if success:
        passes.append(ticker)
    else:
        fails.append(ticker)
        
print('Total passes:', len(passes))
print('Total fails:', len(fails))
    
driver.close()

In [116]:
print('Total passes:', len(passes))
print('Total fails:', len(fails))

Total passes: 9097
Total fails: 7374


In [128]:
passes, fails = list(set(passes)), list(set(fails))

In [126]:
executable_path = 'C:/Users/Fabrice/Documents/GitHub/biz_sentiment/webscrape/drivers/chromedriver.exe'
driver = webdriver.Chrome(executable_path=executable_path)

for _ in range(2):
    for ticker in fails:
        success = mergent_supply_chain_search(driver, ticker)
        if success:
            fails.remove(ticker)
            passes.append(ticker)
        else:
            fails.append(ticker)

KeyboardInterrupt: 

In [130]:
print('Total passes:', len(passes))
print('Total fails:', len(fails))

Total passes: 9798
Total fails: 6667


### Combining all excel files together

In [2]:
import glob
import os
import shutil
import re

In [132]:
paths = glob.glob(r'C:\Users\Fabrice\Downloads\searchresult*')
target_dir = r"C:\Users\Fabrice\Documents\GitHub\biz_sentiment\data\brands\mergent_excels"
for path in paths:
    file_name = os.path.basename(path)
    new_path = os.path.join(target_dir, file_name)
    shutil.move(path, new_path)

In [133]:
new_paths = glob.glob(os.path.join(target_dir, "searchresult*"))
df = pd.concat((pd.read_html(f)[0] for f in new_paths))
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'index': 'brand_num'}, inplace=True)
df['Brands'] = df['Tradename'].str.split(' - ').apply(lambda x: '' if len(x) == 1 else x[-1])

In [134]:
df.to_csv('data/brands/brands_v2.csv')

In [194]:
def preprocess(keyword):
    
    # find all files in Downloads and move them to the project dir
    paths = glob.glob("C:/Users/Fabrice/Downloads/" + keyword + "*")
    target_dir = os.path.join("C:/Users/Fabrice/Documents/GitHub/biz_sentiment/data/brands", keyword)
    os.makedirs(target_dir, exist_ok = True) 
    for path in paths:
        file_name = os.path.basename(path)
        new_path = os.path.join(target_dir, file_name)
        shutil.move(path, new_path)
        
    # load as df    
    new_paths = glob.glob(os.path.join(target_dir, keyword + "*"))
    df = pd.concat((pd.read_html(f)[0] for f in new_paths))
    df.drop_duplicates(inplace=True)
    df.reset_index(inplace=True)
    
    # save as csv
    df.to_csv(os.path.join(target_dir, keyword + '.csv'))

In [195]:
for keyword in ['supplier', 'customer', 'competitor', 'partner']:
    preprocess(keyword)

In [25]:
keyword = 'customer'
target_dir = os.path.join("C:/Users/Fabrice/Documents/GitHub/biz_sentiment/data/brands", keyword)
new_paths = glob.glob(os.path.join(target_dir, keyword + "*"))
# df = pd.concat((pd.read_html(f)[0] for f in new_paths))
# df.drop_duplicates(inplace=True)
# df.reset_index(inplace=True)

In [133]:
for keyword in ['supplier', 'customer', 'competitor', 'partner']:
    
    print(keyword)

    target_dir = os.path.join("C:/Users/Fabrice/Documents/GitHub/biz_sentiment/data/brands", keyword)
    new_paths = glob.glob(os.path.join(target_dir, keyword + "*.xls"))

    # create df
    df = None
    for f in new_paths:
        with open(f, encoding="utf8") as fp:
            soup = BeautifulSoup(fp)
            title = soup.h2.text
        if df is None:
            df = pd.read_html(f)[0]
            df['company'] = title
        else:
            next_df = pd.read_html(f)[0]
            next_df['company'] = title
            df = df.append(next_df)
    # preprocessing
    df['company'] = df['company'].str.replace(' \(New\) ', ' ')
    df['exchange'] = df['company'].apply(lambda x: re.search('\w+:', x).group(0)[:-1])
    df['ticker'] = df['company'].apply(lambda x: re.search(': \w+', x).group(0)[2:])
    df['company'] = df['company'].str.split(' \(\w+: \w+\)').apply(lambda x:x[0])
    df.loc[df['Dir'] == '<', 'Dir'] = 'Others'
    df.loc[df['Dir'] == '>', 'Dir'] = 'Company'
    df.drop_duplicates(inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={'Dir': 'defined_by',
                       'index': keyword + '_num', 
                       'Company': keyword + '_company',
                       'Ticker': keyword + '_ticker'}, inplace=True)
    cols = list(df.columns)
    cols = cols[-3:] + cols[:-3]
    df = df[cols]
    # save to csv
    df.to_csv(os.path.join(target_dir, '..', keyword + '.csv'))

customer
competitor
partner


In [135]:
len(df['company'].unique())

3166

## Spotchecking

https://docs.google.com/spreadsheets/d/12W0NATAeD8twLSGR3S8kYkUUtsVPCGlE443pqw05Nwo/edit#gid=0

In [118]:
import random

In [119]:
random.sample(fails, 20)

['SWIM',
 'TTDKY',
 'CMSB',
 'SPPR',
 'CNVAF',
 'DAVD',
 'BHIX',
 'GAMEE',
 'TRDFF',
 'LBTF',
 'VBTX',
 'STMP',
 'SEGP',
 'CEG',
 'PA',
 'EVVV',
 'SLR',
 'BWLK',
 'UOUT',
 'AMSI']

## Get Competitor, Customer, Supplier, Partner data

In [ ]:
executable_path = 'C:/Users/Fabrice/Documents/GitHub/biz_sentiment/webscrape/drivers/chromedriver.exe'
driver = webdriver.Chrome(executable_path=executable_path)

print('Total tickers:', len(tickers))

found, unfound = set(), tickers
for _ in range(3):
    for ticker in list(unfound):
        success = mergent_basic_search(driver, ticker, skip_inactive=False)
        if success:
            found.union(ticker)
        else:
            unfound.union(ticker)

print('Total found:', len(found))
print('Total unfound:', len(unfound))
    
driver.close()

In [176]:
# driver = webdriver.Chrome(executable_path=executable_path)
# mergent_basic_search(driver, 'BHP', skip_inactive=False)

## Full Selenium Approach (Unstable)

In [ ]:
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# def download_excel(ticker):
#     driver.get("https://www.mergentonline.com/horizonsearch.php")

#     searchbox = driver.find_element_by_id("searchtext")
#     searchbox.clear()
#     searchbox.send_keys(ticker)


#     search_table = WebDriverWait(driver, 3).until(
#         EC.presence_of_element_located(
#             (By.CLASS_NAME, "livesearchresult")
#         )
#     ) 
#     rows = search_table.find_elements(By.TAG_NAME, "tr")
#     for row in rows:
#         driver.implicitly_wait(5)
#         cols = row.find_elements(By.TAG_NAME, "td")
#         try:
#             driver.implicitly_wait(5)
#             link = cols[0].find_elements(By.TAG_NAME, "a")[0].get_attribute("href")
#             driver.implicitly_wait(5)
#             ticker = cols[1].text
#             # print(link, ticker)
#             if 'US' in ticker[-2:]:
#                 driver.get(link)
#         except:
#             driver.implicitly_wait(5)
#             link = cols[0].find_elements(By.TAG_NAME, "a")[0].get_attribute("href")
#             driver.implicitly_wait(5)
#             ticker = cols[1].text
#             # print(link, ticker)
#             if 'US' in ticker[-2:]:
#                 driver.get(link)

#     download_excel = driver.find_element_by_id("excel")
#     download_excel.click()
    
#     driver.close()